<a href="https://colab.research.google.com/github/jdweak/CSE198-CTF/blob/main/Final_CSE198CTF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CTF: Password Cracking

To really understand password security and best practices, in this mock CTF we'll explore and implement password cracking techniques. Our dataset will draw from the OWASP project's dataset of the most common passwords (https://github.com/danielmiessler/SecLists/tree/master).

# Setup
We will use the site https://tinyurl.com/CSE198-Leaderboard to validate flag answers and keep track of team scores. Each flag is worth 100 points, so with 6 flags there is a max score of 600.

Have **1 person only** from your team run the 2 code cells below to add your team to the leaderboard. In the second code cell make sure to set the team_name variable to what you want your team's leaderboard name to be. When submitting answers to the leaderboard using the submit_solution method provided for each flag, have the person who ran those code cells to put your team on the leaderboard do it (requests to the leaderboard without a valid team ID will be rejected with an error).

*If you really want to have multiple people submitting to your team's scoreboard, run the code*

```
user_id = "your team's user id"
# The id is the one printed after team creation, looks something like a65c169b-3669-4bb7-900b-e55b71e9b085
```


In [20]:
# download relavent datasets
!git clone https://github.com/jdweak/CSE198-CTF.git
# download library for talking to leaderboard site
!pip install requests
import requests

# methods for setup
SERVER_URL = "https://cse198.pythonanywhere.com"
user_id = None
def create_user(name):
    global user_id
    response = requests.post(f"{SERVER_URL}/create_user", json={"name": name})
    if response.status_code == 200:
        user_id = response.json()["user_id"]
        print(f"User created! Your team name is {name} and Unique User ID is: {user_id}")
    else:
        print("Failed to create user:", response.json())

def submit_solution(problem_number, solution_text):
    if not user_id:
        print("Error: You must create a user first!")
        return

    payload = {
        "user_id": user_id,
        "problem_number": problem_number,
        "solution": solution_text
    }
    response = requests.post(f"{SERVER_URL}/submit_solution", json=payload)

    if response.status_code == 200:
        print(f"Success! Your new score is {response.json()['new_score']}")
    else:
        print("Failed to submit solution:", response.json())

def check_user_id():
    print(f"Your saved user ID is: {user_id}")



fatal: destination path 'CSE198-CTF' already exists and is not an empty directory.


In [19]:
# Set the team_name variable to what you want your team's name on the leaderboard to be
# and run this cell
team_name = ""
if team_name != "":
  create_user(team_name)
else:
  print("Error: please set the team_name variable and run the cell again")

User created! Your ID is: a65c169b-3669-4bb7-900b-e55b71e9b085


## Flag 1: Finding passwords in a plaintext list

A common security flaw in early websites was storing password databases in plaintext (without hashing). This meant that for any of these websites, if a hacker got access to the database (for example through SQL injection) they could easily find the password of any user regardless of password strength.

For this flag, first run the code snippet below, then look in the plaintext_passwords.csv file (a list of the top 50 most common passwords) and find the password of user 16 (look at files by clicking on the folder symbol on the left of your screen and going into the CSE198-CTF folder). When you find it assign the variable FLAG1 in the cell below to the password and run the cell.

In [25]:
FLAG1 = ""
submit_solution(1, FLAG1)

Failed to submit solution: {'message': 'Problem already solved.'}


## Flag 2: Basic Password Hashing & Dictionary Attack

Flag 1 should have demonstrated why hashing passwords is so important to ensure security. With hashed passwords, instead of seeing a table full of everyone's password, you now see a bunch of long strings which were encrypted with an irreversable hashing algorithm (in this case SHA256). So instead of seeing the plaintext of user 36's password in our table, we now see this:



```
User ID: 16 || SHA256 Hash: 1c8bfe8f801d79745c4631d09fff36c82aa37fc4cce4fc946683d7b336b63032
```

Not so easy to get the credentials now. For this next challenge, we're going to implement a dictionary attack to crack a user's password. In this case, our dictionary will be a subset of the list of the top 1000 most common passwords (passwords 501-950). Use the provided hash_SHA256 method and list of passwords to find the plaintext of the hashed password below:


```
8bd10698229d26627eb039ac20f4537b62c6687ccb2892590bc7a3691659e892
```

Once you figure out the plaintext password, paste it into the FLAG2 variable below like you did in flag 1.


In [ ]:
import hashlib

# This method allows you to hash any string using the SHA256 algorithm.
def hash_SHA256(password):
  sha256_hash = hashlib.sha256(password.encode('utf-8')).hexdigest()
  return sha256_hash

In [ ]:
# Read the password file into a list called plaintexts
with open('/content/CSE198-CTF/plaintext_dictionary.csv.csv', 'r') as f:
    plaintexts = [line.strip() for line in f]

# print out the first plaintext in the list
print(plaintexts[0])


# HINT: If you as a person were given a list of the most common passwords and instructions
# on how to hash a password, how would you do this task by hand?

# Your code goes here


123456


In [ ]:
FLAG2 = "plaintext password"
submit_solution(2, FLAG2)

## Flag 3: Rainbow Table Attack
As we saw in the previous flag, the dictionary attack is a quick and easy way to crack commonly used or leaked passwords. However, in the real world simple dictionary attacks can still be very slow. Many encryption algorithms specifically designed for password hashing like Bcrypt, Argon2, or scrypt are designed to be much slower to improve resistance against password cracking. Additionally, when our dictionary grows from a few hundred passwords to millions, you can see how dictionary attacks can become quite slow.

To fix this, we will implement a rainbow table attack. Rainbow table attacks are similar to dictionary attacks, but instead of taking a list of plaintext passwords and hashing them all in our algorithm we will take a precomputed list of password hashes and their corresponding plaintext passwords and search it for the password we are trying to crack. In this excersise, use the rainbow_table.csv file (mapping the top 500 most common passwords to their SHA256 hashes) to quickly find the password associated with the hash
```
8fced00b6ce281456d69daef5f2b33eaf1a4a29b5923ebe5f1f2c54f5886c7a3
```

*Side note: In the real world rainbow tables are generally a bit more complicated and powerful than we see in this flag. Rainbow tables use a technique called hash chaining to efficiently store a huge amount of hash value -> password pairings, allowing them to cover a much larger subspace of passwords than a simple csv like we do here. Hash chaining is so powerful it allows you to cover all possible passwords as long as the max length isn't too long, essentially precomputing a brute force attack!*

In [ ]:
# Read rainbow table into a python dictionary. The dictionary maps SHA256 hash -> plaintext
import csv
rainbow_table = {}
with open('/content/CSE198-CTF/rainbow_table.csv', mode='r') as f:
    reader = csv.reader(f)
    # skip the header row
    next(reader)
    for row in reader:
        rainbow_table[row[0]] = row[1]

# Your code here

In [ ]:
FLAG3 = "plaintext password"
submit_solution(3, FLAG3)

## Flag 4-6: Brute Force Attacks
Dictionary attacks and rainbow tables are great ways to quickly crack passwords, but what if the hashed password you are trying to crack isn't in any of those lists? We fall back to the attack of last resort: the Brute Force Attack.

In this attack, we will simply try every possible combination of letters and numbers until we find a matching hash value. Theoretically, this attack will always succeed at cracking a password (since all possible combinations are attempted). In practice, strong passwords take so long to crack it's infeasible to actually decipher them using this strategy.

For flags 4-6, use a brute force cracking algorithm to find the plaintext of the following hashes. The length of the plaintext password for some flags will be given to make the coding a bit easier.


Flag 4 (1 point, password length 1 character):


```
acac86c0e609ca906f632b0e2dacccb2b77d22b0621f20ebece1a4835b93f6f0
```



Flag 5 (3 points, password length 3 characters):

```
bf446ea38129b6f1a1ada774c9130c81a33904db1ee88e5b557d76ef1fb5e22d
```


Flag 6 (4 points password length between 1 and 10 characters):


```
31590c8d2e47e8ab4e2b66cf4327748eb9b6abd18fb30c578fb60addf4e1d13d
```


In [ ]:
# All passwords will contain letters only from the below alphabet. A specific letter can be accessed by doing alphabet[idx]
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
print(alphabet[0])

# HINT 1: How would you create a list of all 2 letter combinations after being given a list of all 1 letter combinations?
# How about a list of all 3 letter combinations given the 2 combination list? Try to generalize this concept of building
# combination lists off of previous lists in your code. If you are having trouble thinking about this in code try doing
# it by hand.

# HINT 2: For actually coding this concept, embedded loops or recursion are probably your friends

# HINT 3: The range of possible passwords you are trying for 1-10 characters can get quite large.
# If you run into memory errors think of how you can tweak your code to use less memory (for example,
# trying to make a python list of every possible password probably gets too large, do you need to
# store all that data at a time? If your code is taking too long, consider which ranges of passwords
# are quicker to run through (is it better to check all 10 char passwords first or all 1 char passwords?)

# Your code here


'a'

In [ ]:
FLAG4 = "plaintext password"
submit_solution(4, FLAG4)

In [ ]:
FLAG5 = "plaintext password"
submit_solution(5, FLAG5)

In [ ]:
FLAG6 = "plaintext password"
submit_solution(6, FLAG6)